In [1]:
# RSI PowerZones Weekly
import gta_indicator
import gta_plot
import gta_prices
import gta_statistics
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# Trade parameters.
exchange = 'LSE'
tidm = 'IBPO'
timeframe = 'Weekly'
filename = f'{exchange}_{tidm}_prices.csv'
history_start = '2010-09-14'
p = 48 # Look back period.
strategy = f'Donchian{p}'
position_size = 7500  # Position size in major currency unit.
charges = 11.95  # Commission per trade.
stamp_duty = 0.0  # Stamp Duty percentage.
stop_loss = 20  # Stop Loss percentage.

In [3]:
# Function definitions.
def state_signal(entry_signal, exit_signal, period):
    '''Calculate trade state signals.'''
    df = pd.concat([entry_signal, exit_signal], axis=1)
    df.columns = ['entry', 'exit']
    df['state'] = 0
    for i in range(period, len(df)):
        if df.loc[df.index[i], 'entry'] == 1 \
        and df.loc[df.index[i - 1], 'state'] == 0:
            df.loc[df.index[i], 'state'] = 1
        elif df.loc[df.index[i], 'exit'] == 1:
            df.loc[df.index[i], 'state'] = 0
        else:
            df.loc[df.index[i], 'state'] = df.loc[df.index[i - 1], 'state']
    return df.state


def trade_list(close, entry_flag, exit_flag):
    '''Generate trade list'''
    frame = pd.concat([close, entry_flag, exit_flag], axis=1)
    frame.columns = ['close', 'entry_flag', 'exit_flag']
    entry_price = frame.close[frame.entry_flag == 1]
    exit_price = frame.close[frame.exit_flag == 1]
    entry_count = len(entry_price)
    exit_count = len(exit_price)
    if exit_count != entry_count:
        last_price = pd.Series(data=frame.close[-1:], index=frame.index[-1:])
        exit_price = exit_price.append(last_price)
        exit_count = len(exit_price)
    df = pd.DataFrame(columns=['entry_price', 'exit_date', 'exit_price'])
    df.entry_price = entry_price
    df.exit_date = exit_price.index
    df.exit_price = exit_price.values
    return df

In [4]:
# Import weekly closing prices.
prices = gta_prices.weekly(exchange, tidm, history_start)

In [5]:
# Calculate Donchian channel.
df = gta_indicator.donchian(prices, p)

In [6]:
# Check for close above upper Donchian channel.
df['bto'] = np.where(df.close > df.upr, 1, 0)

# Check for close below mid Donchian channel.
df['stc'] = np.where(df.close < df.mid, 1, 0)

# Trade state signal.
df['ss'] = state_signal(df.bto, df.stc, p)
pd.set_option('display.max_rows', None)

# Trade entry & exit flags.
df['en1'] = np.where(np.logical_and(df.ss == 1, df.ss.shift(periods=1) == 0), 1, 0)
df['ex1'] = np.where(np.logical_and(df.ss == 0, df.ss.shift(periods=1) == 1), 1, 0)

# df.loc['2018':'2021']

In [7]:
# Trade list.
td = trade_list(df.close, df.en1, df.ex1)  # Trade List #1.
td = td.sort_index()  # Sort trades in ascending date order.
td['weeks'] = (td.exit_date - td.index) / np.timedelta64(1, 'W')
td['chg_pct'] = ((td.exit_price - td.entry_price) / td.entry_price) * 100
td['charges'] = charges
td['stamp_duty'] = position_size * (stamp_duty / 100)
td['profit'] = position_size * (td.chg_pct / 100) - td.charges - td.stamp_duty
td['cum_profit'] = td.profit.cumsum()
cum_profit = td.cum_profit
td.round(2)

,entry_price,exit_date,exit_price,weeks,chg_pct,charges,stamp_duty,profit,cum_profit
date,,,,,,,,,
2011-08-26,2.26,2013-08-02,3.51,101.0,55.31,11.95,0.0,4136.28,4136.28
2016-05-13,0.42,2017-07-21,0.66,62.0,57.14,11.95,0.0,4273.76,8410.04
2018-09-07,0.82,2020-02-28,2.17,77.0,163.03,11.95,0.0,12215.32,20625.37
2020-11-20,2.88,2021-05-07,2.97,24.0,3.13,11.95,0.0,222.43,20847.79
2022-04-15,4.23,2022-04-22,4.28,1.0,1.18,11.95,0.0,76.70,20924.49
